In [73]:
#from google.colab import drive
#drive.mount("/content/drive")

In [74]:
!pip install rdflib
!pip install pydicom
from rdflib import *
from hashlib import sha256


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Create Source Graph

In [75]:
schema = Graph()
schema.bind('tb', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
schema.bind('ab', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
dqv = Namespace('http://www.w3.org/ns/dqv#')

## Load Data

In [76]:
import pandas as pd

In [77]:
import pydicom

In [78]:
ds = pydicom.dcmread('/home/acraf/psr/tfm/Fdatavalidation/DataProduct/Data/Explotation/0002.DCM')

# Print some header information
print("Patient's Name: ", ds.PatientName)
print("Patient ID: ", ds.PatientID)
print("Modality: ", ds.Modality)
print("Study Date: ", ds.StudyDate)

Patient's Name:  Rubo DEMO
Patient ID:  556342B
Modality:  XA
Study Date:  19941013


## Data Product Profile

### OWNER

In [79]:
metadata_path = "/home/acraf/psr/tfm/Fdatavalidation/DataProduct/Metadata/explotaiton.csv"

In [80]:
meta_df = pd.read_csv(metadata_path)
meta_dp = meta_df.loc[meta_df['name'] == "UPENN-GBM_clinical_info_v2"]
owner = meta_dp.owner.values[0]


### NAME

In [81]:
dataset_name = "dicomimage"
dataset_name

'dicomimage'

add triples

In [82]:
# Name
schema.add((abox[dataset_name], RDF.type, dcat.Dataset))
#identifier
identifier = sha256(dataset_name.encode('utf-8')).hexdigest()
schema.add((abox[dataset_name],dcterms.identifier, Literal(identifier)))
#owner
schema.add((abox[dataset_name], tbox.owner, Literal(owner)))




<Graph identifier=N9d22fb4f3b354c67affc41b97b1c4f3c (<class 'rdflib.graph.Graph'>)>

### ATTRIBUTES


In [83]:
# Iterate over all attributes
for attribute in dir(ds)[:50]:
    # Check if the attribute starts with a letter
    # This is to filter out Python built-in attributes
    if attribute[0].isalpha():
        # Check if the attribute exists in the DICOM file
        if hasattr(ds, attribute):
            # Create RDF triple for Attribute
            schema.add((abox[attribute], RDF.type, tbox.Attribute))
            # Create RDF triple for Attribute
            schema.add((abox[dataset_name], tbox.hasAttribute, abox[attribute]))
            # rdf for the dataproperty
            schema.add((abox[attribute], tbox.attribute, Literal(attribute)))



## DatasetTypeTemplate

### Format

In [84]:
format = 'dicom'

In [85]:
# triple
schema.add((abox.Image, RDF.type, tbox.DatasetTypeTemplate))

# format
schema.add((abox.Image, dcterms['format'], Literal(format)))  # Correct usage of the namespace

# addDTT
schema.add((abox[dataset_name], tbox.hasDTT, abox.Image))


<Graph identifier=N9d22fb4f3b354c67affc41b97b1c4f3c (<class 'rdflib.graph.Graph'>)>

### TechnologyAspects

In [86]:
#triple
schema.add((abox.TA1, RDF.type, tbox.TechnologyAspects))

#  addTA
schema.add((abox[dataset_name], tbox.hasTA, abox.TA1))

#TYPE
schema.add((abox.TA1, tbox.typeAcces, abox.Static))
schema.add((abox.Static, RDF.type, tbox.Acces))

# PATH
data_path = "/home/acraf/psr/tfm/Fdatavalidation/DataProduct/Data/Explotation/0002.DCM"
schema.add((abox.Static, tbox.path, Literal(data_path)))


<Graph identifier=N9d22fb4f3b354c67affc41b97b1c4f3c (<class 'rdflib.graph.Graph'>)>

# SAVE SOURCE_GRAPH

In [87]:
rdf_format = "turtle"
schema.serialize(destination="./schema_metadata_dicom.ttl", format=rdf_format)

<Graph identifier=N9d22fb4f3b354c67affc41b97b1c4f3c (<class 'rdflib.graph.Graph'>)>